In [44]:
import PIL
import torch

import pickle
import numpy as np
import pandas as pd 
from sklearn.metrics import f1_score
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision import transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader



import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import models
import time
import copy
from tqdm.autonotebook import tqdm, trange

from matplotlib import colors, pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
import zipfile
!chcp 65001

Active code page: 65001


In [45]:
#Почистим кэш
torch.cuda.empty_cache()

In [46]:
TRAIN_DIR = Path('Data_unzip/train/simpsons_dataset')
TEST_DIR = Path('Data_unzip/testset/testset')
train_val_files = sorted(list(TRAIN_DIR.rglob('*.jpg')))
test_files = sorted(list(TEST_DIR.rglob('*.jpg')))
# разные режимы датасета 
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE = 224
# работаем на видеокарте
DEVICE = torch.device("cuda")

class SimpsonsDataset_Flip(Dataset):
    """
    Датасет с картинками, который паралельно подгружает их из папок
    производит скалирование и превращение в торчевые тензоры
    """
    def __init__(self, files, mode):
        super().__init__()
        # список файлов для загрузки
        self.files = sorted(files)
        # режим работы
        self.mode = mode
        self.idx = None

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.files)
     
        self.label_encoder = LabelEncoder()

        if self.mode != 'test':
            self.labels = [path.parent.name for path in self.files]
            self.label_encoder.fit(self.labels)
            self.idx = self.label_encoder.transform(self.labels)
            with open('label_encoder.pkl', 'wb') as le_dump_file:
                  pickle.dump(self.label_encoder, le_dump_file)
                      
    def __len__(self):
        return self.len_
      
    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image
  
    def __getitem__(self, index):
        # для преобразования изображений в тензоры PyTorch и нормализации входа
        transform = {
            'train': transforms.Compose([
                transforms.ToTensor(),
                transforms.RandomHorizontalFlip(0.25),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'val': transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'test': transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        }
        
        x = self.load_sample(self.files[index])
        x = self._prepare_sample(x)
        x = np.array(x / 255, dtype='float32')
        x = transform[self.mode](x)
        if self.mode == 'test':
            return x
        else:
            label = self.labels[index]
            label_id = self.label_encoder.transform([label])
            y = label_id.item()
            return x, y
        
    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
        return np.array(image)

In [47]:
from sklearn.model_selection import train_test_split
train_val_labels = [path.parent.name for path in train_val_files]
train_files, val_files = train_test_split(train_val_files, test_size=0.25, \
                                          stratify=train_val_labels)

In [48]:
def generate_aug(train_dataset, N_aug = 100):
    df = pd.DataFrame()
    df['name'] = train_dataset.labels
    df['Id'] = train_dataset.idx
    table = df['Id'].value_counts()
    
    I = []
    nn = []
    A = 0
    L = []
    OST = []
    for i in range(len(train_dataset.idx)):
        if(table[train_dataset.idx[i]]<N_aug):
            #Зашли первый раз!
            A = A+1
            if(A==1): 
                tmp = train_dataset.labels[i]
                nn.append(train_dataset.files[i])
                L.append(train_dataset.idx[i])
            if(A>1):
                if(tmp == train_dataset.labels[i]):
                    nn.append(train_dataset.files[i])
                    if((tmp == 'troy_mcclure') and (len(nn)==6)):
                        I.append(nn)

                else:
                    I.append(nn)
                    nn = [train_dataset.files[i]]
                    tmp = train_dataset.labels[i]
                    L.append(train_dataset.idx[i])
        else:
            OST.append(train_dataset.files[i])
        
    s = 0
    for i in range(len(I)):
        s += len(I[i])

    if((s!=904) and (N_aug == 100)):
        print("Ошибка ! Не хватает ссылок на картинки !")
        return None
    
    else:
        for i in range(len(I)):
            I[i] += list(np.random.choice(I[i], size = N_aug-len(I[i])))
        
        Z = [num for elem in I for num in elem]
        end = OST+Z
        return end 


val_dataset = SimpsonsDataset_Flip(val_files, mode='val')
train_dataset = SimpsonsDataset_Flip(train_files, mode='train')
train_aug_dataset = SimpsonsDataset_Flip(generate_aug(train_dataset, N_aug = 500), mode='train')

In [49]:
#Сделаем 3 дата лоудера

batch_size = 20
train_loader_aug = DataLoader(train_aug_dataset, batch_size=batch_size, shuffle = True)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = True)

In [50]:
# Очень простая сеть
class SimpleCnn(nn.Module):
  
    def __init__(self, n_classes):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.BatchNorm2d(8)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.BatchNorm2d(16)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.BatchNorm2d(32)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.BatchNorm2d(64)
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=96, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.BatchNorm2d(96)
        )
        
        self.out = nn.Linear(96 * 5 * 5, n_classes)
  
  
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)

        x = x.view(x.size(0), -1)
        logits = self.out(x)
        return logits


#Обучение
def fit_epoch(model, train_loader, criterion, optimizer, scheduler):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    for inputs, labels in train_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)
    
    scheduler.step()         
    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    return train_loss, train_acc

def eval_epoch(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    for inputs, labels in val_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
    val_loss = running_loss / processed_size
    val_acc = running_corrects.double() / processed_size
    return val_loss, val_acc


def train(train_loader, val_loader, model, epochs, parametrs):

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        opt = torch.optim.Adam(parametrs, lr = 2e-4)
        criterion = nn.CrossEntropyLoss()
        exp_lr_scheduler = lr_scheduler.StepLR(opt, step_size=16, gamma=0.1)

        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt, exp_lr_scheduler)
            print("loss", train_loss)
            
            val_loss, val_acc = eval_epoch(model, val_loader, criterion)
            history.append((train_loss, train_acc, val_loss, val_acc))
            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))
            
    return history


def predict(model, test_loader):
    with torch.no_grad():
        logits = []
    
        for inputs in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [54]:
def predict_one_sample(model, inputs, device=DEVICE):
    """Предсказание, для одной картинки"""
    with torch.no_grad():
        inputs = inputs.to(device)
        model.eval()
        logit = model(inputs).cpu()
        probs = torch.nn.functional.softmax(logit, dim=-1).numpy()
    return probs

def F1_score(val_dataset, model_extractor):
    imgs = [val_dataset[id][0].unsqueeze(0) for id in range(len(val_dataset))]
    probs_ims = predict(model_extractor, imgs)
    y_pred = np.argmax(probs_ims,-1)
    actual_labels = [val_dataset[id][1] for id in range(len(val_dataset))]
    return (f1_score(actual_labels, y_pred, average='weighted'))

In [55]:
n_classes = len(np.unique(train_val_labels))
PATH1 = "CNN_weights/weights_CNN_aug100.pt"
PATH2 = "CNN_weights/weights_CNN_aug500_flip_shed.pt"
PATH3 = "CNN_weights/weights_CNN_aug500_flip_shed_new.pt"

simple_cnn_first = SimpleCnn(n_classes).to(DEVICE)
simple_cnn_second = SimpleCnn(n_classes).to(DEVICE)
simple_cnn_third = SimpleCnn(n_classes).to(DEVICE)

# загружаем сохраненное состояние весов нейросети
simple_cnn_first.load_state_dict(torch.load(PATH1))
simple_cnn_second.load_state_dict(torch.load(PATH2))
simple_cnn_third.load_state_dict(torch.load(PATH3))

<All keys matched successfully>

In [56]:
%%time
F1_score(val_dataset, simple_cnn_first), F1_score(val_dataset, simple_cnn_second), F1_score(val_dataset, simple_cnn_third)

(0.9403116949549345, 0.9597637240798483, 0.9611922970060328)

### Очень запомним эти скоры, сейчас захуярим Alex_net и проверим как будет работать, нам надо 97!!!

In [61]:
#Зафигачу сейчас веса, которые получу с кагла для Alex
PATH_alex_net_17 = "Alex_Net_weights/weights_Alex_Net17_epochs.pt"
PATH_alex_net_19 = "Alex_Net_weights/weights_Alex_Net19_epochs.pt"

In [59]:
#Пробуем AlexNet
%time
DEVICE = torch.device("cuda")
model_extractor = models.alexnet(pretrained=True)

Wall time: 997 µs


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\Николай/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth


In [60]:
layers_to_unfreeze = 5
# Выключаем подсчет градиентов для слоев, которые не будем обучать
for param in model_extractor.features[0:layers_to_unfreeze].parameters():
    param.requires_grad = False

In [62]:
# num_features -- это размерность вектора фич, поступающего на вход FC-слою
num_features = 9216
# Заменяем Fully-Connected слой на наш линейный классификатор
model_extractor.classifier = nn.Linear(num_features, n_classes)
model_extractor = model_extractor.to(DEVICE)

In [63]:
# загружаем сохраненное состояние весов нейросети
model_extractor.load_state_dict(torch.load(PATH_alex_net_19))

<All keys matched successfully>

In [64]:
%%time
F1_score(val_dataset, model_extractor)

Wall time: 4min 58s


0.9610696823354545